# Crawler and indicators
- Collect SP500 stock info from Wikipedia from Wikipedia [List of S&P 500 companies](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)
- Scrape [Income Statement](https://www.marketwatch.com/investing/stock/aapl/financials), and [Balance Sheet](https://www.marketwatch.com/investing/stock/aapl/financials/balance-sheet) 
- Calculate indicators (eps, eps growth, net income)

In [1]:
# Import Packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date
import formats as format
import requests

In [7]:
# Getting SP500 stocks info from wikipedia
print("Getting SP500 stocks info from wikipedia")
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
stocks_info=[]
tickers = []
securities = []
gics_industries = []
gics_sub_industries = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    security = row.findAll('td')[1].text
    gics_industry = row.findAll('td')[3].text
    gics_sub_industry = row.findAll('td')[4].text

    tickers.append(ticker.lower().replace(r"\n", " "))
    securities.append(security)
    gics_industries.append(gics_industry.lower())
    gics_sub_industries.append(gics_sub_industry.lower())

stocks_info.append(tickers)
stocks_info.append(securities)
stocks_info.append(gics_industries)
stocks_info.append(gics_sub_industries)

stocks_info_df = pd.DataFrame(stocks_info).T
stocks_info_df.columns=['tickers','security','gics_industry','gics_sub_industry']
stocks_info_df['seclabels'] = 'SP500'
stocks_info_df['labels'] = stocks_info_df[['tickers','security', 'gics_industry','gics_sub_industry','seclabels']].apply(lambda x: ' '.join(x), axis=1)
stocks_info_df['tickers']= stocks_info_df['tickers'].apply(lambda x: x[:-1])

Getting SP500 stocks info from wikipedia


In [80]:
stocks_info_df.head(3)

,tickers,security,gics_industry,gics_sub_industry,seclabels,labels
0,mmm,3M,industrials,industrial conglomerates,SP500,mmm\n 3M industrials industrial conglomerates ...
1,aos,A. O. Smith,industrials,building products,SP500,aos\n A. O. Smith industrials building product...
2,abt,Abbott,health care,health care equipment,SP500,abt\n Abbott health care health care equipment...


In [11]:
# Create a list of dict based on tickers and labels
dictlist = []
for index, row in stocks_info_df.iterrows():
    dictlist.append({'value':row['tickers'], 'label':row['labels']})

In [12]:
# function get element in list 
def getelementinlist(list,element):
    try:
        return list[element]
    except:
        return '-'

### Scrape [Income Statement](https://www.marketwatch.com/investing/stock/aapl/financials), and [Balance Sheet](https://www.marketwatch.com/investing/stock/aapl/financials/balance-sheet) 
- Calculate indicators (eps, eps growth, net income) 

In [61]:
def getfinancialreportingdf(ticker):

    # try:
    urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
    urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'
    text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"lxml") #read in
    text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"lxml") #read in


    # Income statement crawler:
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'overflow__cell fixed--column'})
    epslist=[]
    netincomelist = []
    longtermdebtlist = [] 
    interestexpenselist = []
    ebitdalist= []

    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])


    # Balance sheet crawler:
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'overflow__cell fixed--column'})
    equitylist=[]
    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'overflow__cell'}) if td.text])

    # Calculate indicators:

    eps = getelementinlist(epslist,0)
    epsgrowth = getelementinlist(epslist,1)
    netincome = getelementinlist(netincomelist,0)
    shareholderequity = getelementinlist(equitylist,0)
    roa = getelementinlist(equitylist,1)

    longtermdebt = getelementinlist(longtermdebtlist,0)
    interestexpense =  getelementinlist(interestexpenselist,0)
    ebitda = getelementinlist(ebitdalist,0)

    ## Make it into Dataframes and reformat

    df= pd.DataFrame({'eps': eps,'epsgrowth': epsgrowth,'netincome': netincome,'shareholderequity': shareholderequity,'roa': 
                  roa,'longtermdebt': longtermdebt,'interestexpense': interestexpense,'ebitda': ebitda},index=['2022','2018','2019','2020','2021',
'5-YEAR TREND'])
    df = df.head(5).sort_index(ascending=True)

    return df

In [81]:
df=getfinancialreportingdf('AAPL')

In [52]:
df

,eps,epsgrowth,netincome,shareholderequity,roa,longtermdebt,interestexpense,ebitda
2022,6.15,-,99.8B,50.67B,14.36%,109.71B,2.93B,130.54B
2018,3,-51.22%,59.53B,107.15B,29.30%,93.74B,3.24B,80.34B
2019,2.99,-0.37%,55.26B,90.49B,26.73%,91.81B,3.58B,74.54B
2020,3.31,10.60%,57.41B,65.34B,20.17%,107.05B,2.87B,76.4B
2021,5.67,71.34%,94.68B,63.09B,17.97%,119.38B,2.65B,120.23B


### Add in roe, interest coverage ratio

In [77]:
# Convert text with multipliers to number
multipliers = {'K':1000, 'M':1000000, 'B':1000000000}

def string_to_int(string):
    if string[-1].isdigit(): # check if no suffix
        return int(string)
    mult = multipliers[string[-1]] # look up suffix to get multiplier
     # convert number to float, multiply by multiplier, then make int
    return int(float(string[:-1]) * mult)


# Getting financial reporting with roe, interestcoverageratio
def getfinancialreportingdfformatted(ticker):
    df = getfinancialreportingdf(ticker)

    # Adding roe, interest coverage ratio by convert text to number
    df['roe'] = df.netincome.apply(string_to_int)/df.shareholderequity.apply(string_to_int)
    df['interestcoverageratio'] = df.ebitda.apply(string_to_int)/df.interestexpense.apply(string_to_int)

    # Round to 2 decimals
    df['roe']=df['roe'].apply(lambda x: "{:.2%}".format(x))
    df['interestcoverageratio']=df['interestcoverageratio'].apply(lambda x: round(x,2))
#     Insert ticker and df
    return df

In [78]:
dfformatted = getfinancialreportingdfformatted('AAPL')


In [79]:
dfformatted

,eps,epsgrowth,netincome,shareholderequity,roa,longtermdebt,interestexpense,ebitda,roe,interestcoverageratio
2018,3,-51.22%,59.53B,107.15B,29.30%,93.74B,3.24B,80.34B,55.56%,24.80
2019,2.99,-0.37%,55.26B,90.49B,26.73%,91.81B,3.58B,74.54B,61.07%,20.82
2020,3.31,10.60%,57.41B,65.34B,20.17%,107.05B,2.87B,76.4B,87.86%,26.62
2021,5.67,71.34%,94.68B,63.09B,17.97%,119.38B,2.65B,120.23B,150.07%,45.37
2022,6.15,-,99.8B,50.67B,14.36%,109.71B,2.93B,130.54B,196.96%,44.55
